In [1]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers,Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPool2D,Dropout
from tensorflow.keras.callbacks import EarlyStopping
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import accuracy_score
import pathlib
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
import tensorflow_datasets as tfds
imdb, info = tfds.load("imdb_reviews/subwords8k", with_info=True, as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling C:\Users\shaik\tensorflow_datasets\imdb_reviews\subwords8k\1.0.0.incompleteM8DDF6\imdb_reviews-train…

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling C:\Users\shaik\tensorflow_datasets\imdb_reviews\subwords8k\1.0.0.incompleteM8DDF6\imdb_reviews-test.…

Generating unsupervised examples...: 0 examples [00:00, ? examples/s]

Shuffling C:\Users\shaik\tensorflow_datasets\imdb_reviews\subwords8k\1.0.0.incompleteM8DDF6\imdb_reviews-unsup…

Dataset imdb_reviews downloaded and prepared to C:\Users\shaik\tensorflow_datasets\imdb_reviews\subwords8k\1.0.0. Subsequent calls will reuse this data.


In [10]:
class LearningRateReducerCb(tf.keras.callbacks.Callback):

  def on_epoch_end(self, epoch, logs={}):
    old_lr = self.model.optimizer.lr.read_value()
    new_lr = old_lr * 0.99
    print("\nEpoch: {}. Reducing Learning Rate from {} to {}".format(epoch, old_lr, new_lr))
    self.model.optimizer.lr.assign(new_lr)
    
def create_checkpoint_callback(model_name):
  return tf.keras.callbacks.ModelCheckpoint(filepath=f"{model_name}/checkpoint.ckpt",
                                                          #  monitor="val_accuracy",
                                                           save_best_only=True,
                                                           save_weights_only=True,
                                                           save_freq="epoch")

es_cb = EarlyStopping(monitor='val_loss', patience=10)

In [2]:
train_data, test_data = imdb['train'], imdb['test']

In [4]:
tokenizer = info.features['text'].encoder

In [5]:
sample_string = 'TensorFlow, from basics to mastery'

tokenized_string = tokenizer.encode(sample_string)
print ('Tokenized string is {}'.format(tokenized_string))

original_string = tokenizer.decode(tokenized_string)
print ('The original string: {}'.format(original_string))


Tokenized string is [6307, 2327, 4043, 2120, 2, 48, 4249, 4429, 7, 2652, 8050]
The original string: TensorFlow, from basics to mastery


In [8]:
BUFFER_SIZE = 1000

train_batches = (
    train_data
    .shuffle(BUFFER_SIZE)
    .padded_batch(32))

test_batches = (
    test_data
    .padded_batch(32))

In [12]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(tokenizer.vocab_size, 64),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(),
    metrics=["accuracy"]
)
# Fit the model
model_history = model.fit(train_batches,
                              validation_data=test_batches,
                              epochs=10,
                              callbacks=[create_checkpoint_callback(model.name)])

Epoch 1/10
739/782 [===========================>..] - ETA: 53s - loss: 0.6005 - accuracy: 0.6684

KeyboardInterrupt: 

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(tokenizer.vocab_size, 64),
    tf.keras.layers.Conv1D(128, 5, activation='relu'),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(),
    metrics=["accuracy"]
)
# Fit the model
model_history = model.fit(train_batches,
                              validation_data=test_batches,
                              epochs=10,
                              callbacks=[create_checkpoint_callback(model.name)])

In [ ]:
embedding_dim = 64

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(tokenizer.vocab_size, embedding_dim),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(),
    metrics=["accuracy"]
)
# Fit the model
model_history = model.fit(train_batches,
                              validation_data=test_batches,
                              epochs=10,
                              callbacks=[create_checkpoint_callback(model.name)])

# Json

In [16]:
data=pd.read_json('Sarcasm_Headlines_Dataset.json',lines=True)
data

,article_link,headline,is_sarcastic
0,https://www.huffingtonpost.com/entry/versace-b...,former versace store clerk sues over secret 'b...,0
1,https://www.huffingtonpost.com/entry/roseanne-...,the 'roseanne' revival catches up to our thorn...,0
2,https://local.theonion.com/mom-starting-to-fea...,mom starting to fear son's web series closest ...,1
3,https://politics.theonion.com/boehner-just-wan...,"boehner just wants wife to listen, not come up...",1
4,https://www.huffingtonpost.com/entry/jk-rowlin...,j.k. rowling wishes snape happy birthday in th...,0
...,...,...,...
26704,https://www.huffingtonpost.com/entry/american-...,american politics in moral free-fall,0
26705,https://www.huffingtonpost.com/entry/americas-...,america's best 20 hikes,0
26706,https://www.huffingtonpost.com/entry/reparatio...,reparations and obama,0
26707,https://www.huffingtonpost.com/entry/israeli-b...,israeli ban targeting boycott supporters raise...,0


In [17]:
data.drop(['article_link'],axis=1,inplace=True)
data

,headline,is_sarcastic
0,former versace store clerk sues over secret 'b...,0
1,the 'roseanne' revival catches up to our thorn...,0
2,mom starting to fear son's web series closest ...,1
3,"boehner just wants wife to listen, not come up...",1
4,j.k. rowling wishes snape happy birthday in th...,0
...,...,...
26704,american politics in moral free-fall,0
26705,america's best 20 hikes,0
26706,reparations and obama,0
26707,israeli ban targeting boycott supporters raise...,0


In [18]:
data_shuffled = data.sample(frac=1, random_state=42) # shuffle with random_state=42 for reproducibility
data_shuffled.head()

,headline,is_sarcastic
6913,isis recruiter excited to be talking to popula...,1
11611,jimmy fallon could barely keep it together dur...,0
25947,drunk ron weasley wishing harry potter 'happy ...,0
24478,hoosier hostility: not the american way,0
11863,turns out running doesn't wreck your knees aft...,0


In [19]:
train_sentences, val_sentences, train_labels, val_labels = train_test_split(data_shuffled["headline"].to_numpy(),
                                                                            data_shuffled["is_sarcastic"].to_numpy(),
                                                                            test_size=0.2,
                                                                            random_state=42)

len(train_sentences), len(val_sentences), len(train_labels), len(val_labels)

(21367, 5342, 21367, 5342)

In [21]:
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

10

In [60]:
# Setup text vectorization with custom variables
max_vocab_length = 1000 # max number of words to have in our vocabulary
max_length = 10 # max length our sequences will be (e.g. how many words from a Tweet does our model see?)
text_vectorizer = TextVectorization(max_tokens=max_vocab_length, # how many words in the vocabulary (all of the different words in your text)
                                    standardize="lower_and_strip_punctuation", # how to process text
                                    split="whitespace", # how to split tokens
                                    ngrams=None, # create groups of n-words?
                                    output_mode="int", # how to map tokens to numbers
                                    output_sequence_length=max_length) # how long should the output sequence of tokens be?
                                    # pad_to_max_tokens=True) # Not valid if using max_tokens=None

In [61]:
text_vectorizer.adapt(train_sentences)

In [62]:
embedding = layers.Embedding(input_dim=max_vocab_length, # set input shape
                             output_dim=128, # set size of embedding vector
                             embeddings_initializer="uniform", # default, intialize randomly
                             input_length=max_length, # how long is each input
                             name="embedding_1") 

In [65]:
inputs = layers.Input(shape=(1,), dtype=tf.string, name="input_layer")
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.LSTM(64, return_sequences=True)(x)
x = layers.LSTM(64)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = tf.keras.Model(inputs, outputs, name="model_LSTM")
# Compile the model
model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(),
    metrics=["accuracy"]
)
# Fit the model
model_history = model.fit(train_sentences,
                              train_labels,
                              validation_data=(val_sentences, val_labels),
                              epochs=10,
                              callbacks=[create_checkpoint_callback(model.name),LearningRateReducerCb()])

Epoch 1/10
665/668 [============================>.] - ETA: 0s - loss: 0.3351 - accuracy: 0.8512
Epoch: 0. Reducing Learning Rate from 0.0010000000474974513 to 0.0009900000877678394
668/668 [==============================] - 13s 16ms/step - loss: 0.3351 - accuracy: 0.8512 - val_loss: 0.4071 - val_accuracy: 0.8270
Epoch 2/10
667/668 [============================>.] - ETA: 0s - loss: 0.2689 - accuracy: 0.8787
Epoch: 1. Reducing Learning Rate from 0.0009900000877678394 to 0.000980100128799677
668/668 [==============================] - 10s 15ms/step - loss: 0.2689 - accuracy: 0.8787 - val_loss: 0.4336 - val_accuracy: 0.8179
Epoch 3/10
665/668 [============================>.] - ETA: 0s - loss: 0.2441 - accuracy: 0.8910
Epoch: 2. Reducing Learning Rate from 0.000980100128799677 to 0.0009702991228550673
668/668 [==============================] - 10s 15ms/step - loss: 0.2441 - accuracy: 0.8910 - val_loss: 0.4543 - val_accuracy: 0.8205
Epoch 4/10
668/668 [==============================] - ETA: 0

In [47]:
inputs = layers.Input(shape=(1,), dtype=tf.string, name="input_layer")
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.Conv1D(128, 5, activation='relu')(x)
x = layers.GlobalAveragePooling1D()(x)
x= layers.Dense(64, activation='relu')(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = tf.keras.Model(inputs, outputs, name="model_LSTM")

# Compile the model
model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(),
    metrics=["accuracy"]
)
# Fit the model
model_history = model.fit(train_sentences,
                              train_labels,
                              validation_data=(val_sentences, val_labels),
                              epochs=10,
                              callbacks=[create_checkpoint_callback(model.name)])

Epoch 1/10
668/668 [==============================] - 7s 9ms/step - loss: 0.3669 - accuracy: 0.8339 - val_loss: 0.4051 - val_accuracy: 0.8091
Epoch 2/10
668/668 [==============================] - 6s 9ms/step - loss: 0.3151 - accuracy: 0.8581 - val_loss: 0.4050 - val_accuracy: 0.8179
Epoch 3/10
668/668 [==============================] - 6s 9ms/step - loss: 0.2842 - accuracy: 0.8734 - val_loss: 0.4289 - val_accuracy: 0.8184
Epoch 4/10
668/668 [==============================] - 6s 9ms/step - loss: 0.2530 - accuracy: 0.8912 - val_loss: 0.4404 - val_accuracy: 0.8160
Epoch 5/10
668/668 [==============================] - 6s 9ms/step - loss: 0.2235 - accuracy: 0.9052 - val_loss: 0.5132 - val_accuracy: 0.8074
Epoch 6/10
668/668 [==============================] - 5s 8ms/step - loss: 0.1901 - accuracy: 0.9191 - val_loss: 0.5537 - val_accuracy: 0.8063
Epoch 7/10
668/668 [==============================] - 6s 9ms/step - loss: 0.1632 - accuracy: 0.9319 - val_loss: 0.6238 - val_accuracy: 0.8034
Epoch 

In [51]:
inputs = layers.Input(shape=(1,), dtype=tf.string, name="input_layer")
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GlobalAveragePooling1D()(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = tf.keras.Model(inputs, outputs, name="model_LSTM")

# Compile the model
model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(),
    metrics=["accuracy"]
)
# Fit the model
model_history = model.fit(train_sentences,
                              train_labels,
                              validation_data=(val_sentences, val_labels),
                              epochs=10,
                              callbacks=[create_checkpoint_callback(model.name),es_cb,LearningRateReducerCb()])

Epoch 1/10
660/668 [============================>.] - ETA: 0s - loss: 0.4770 - accuracy: 0.7879
Epoch: 0. Reducing Learning Rate from 0.0010000000474974513 to 0.0009900000877678394
668/668 [==============================] - 5s 6ms/step - loss: 0.4763 - accuracy: 0.7885 - val_loss: 0.3700 - val_accuracy: 0.8439
Epoch 2/10
664/668 [============================>.] - ETA: 0s - loss: 0.3111 - accuracy: 0.8713
Epoch: 1. Reducing Learning Rate from 0.0009900000877678394 to 0.000980100128799677
668/668 [==============================] - 4s 6ms/step - loss: 0.3109 - accuracy: 0.8714 - val_loss: 0.3556 - val_accuracy: 0.8446
Epoch 3/10
660/668 [============================>.] - ETA: 0s - loss: 0.2661 - accuracy: 0.8921
Epoch: 2. Reducing Learning Rate from 0.000980100128799677 to 0.0009702991228550673
668/668 [==============================] - 4s 6ms/step - loss: 0.2663 - accuracy: 0.8921 - val_loss: 0.3646 - val_accuracy: 0.8448
Epoch 4/10
663/668 [============================>.] - ETA: 0s - lo

In [67]:
inputs = layers.Input(shape=(1,), dtype=tf.string, name="input_layer")
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True))(x)
x = layers.Bidirectional(tf.keras.layers.LSTM(32))(x)
x = layers.Dense(64, activation="relu")(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = tf.keras.Model(inputs, outputs, name="model_LSTM")

# Compile the model
model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(),
    metrics=["accuracy"]
)
# Fit the model
model_history = model.fit(train_sentences,
                              train_labels,
                              validation_data=(val_sentences, val_labels),
                              epochs=10,
                              callbacks=[create_checkpoint_callback(model.name),es_cb,LearningRateReducerCb()])

Epoch 1/10
668/668 [==============================] - ETA: 0s - loss: 0.3087 - accuracy: 0.8639
Epoch: 0. Reducing Learning Rate from 0.0010000000474974513 to 0.0009900000877678394
668/668 [==============================] - 21s 26ms/step - loss: 0.3087 - accuracy: 0.8639 - val_loss: 0.4552 - val_accuracy: 0.7956
Epoch 2/10
666/668 [============================>.] - ETA: 0s - loss: 0.2346 - accuracy: 0.8968
Epoch: 1. Reducing Learning Rate from 0.0009900000877678394 to 0.000980100128799677
668/668 [==============================] - 16s 24ms/step - loss: 0.2344 - accuracy: 0.8969 - val_loss: 0.4752 - val_accuracy: 0.8184
Epoch 3/10
667/668 [============================>.] - ETA: 0s - loss: 0.1976 - accuracy: 0.9150
Epoch: 2. Reducing Learning Rate from 0.000980100128799677 to 0.0009702991228550673
668/668 [==============================] - 16s 24ms/step - loss: 0.1975 - accuracy: 0.9150 - val_loss: 0.5484 - val_accuracy: 0.8152
Epoch 4/10
667/668 [============================>.] - ETA: 0

In [28]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, None, 64)          523840    
                                                                 
 global_average_pooling1d (G  (None, 64)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dense_6 (Dense)             (None, 6)                 390       
                                                                 
 dense_7 (Dense)             (None, 1)                 7         
                                                                 
Total params: 524,237
Trainable params: 524,237
Non-trainable params: 0
_________________________________________________________________
